In [ ]:
## import libraries
import os, sys
import yaml
import pandas as pd
import numpy as np
%matplotlib inline

# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature

# Import my modules
sys.path.append('../modules') # Path to modules
import ar_funcs
from plotter import draw_basemap

# import personal modules
sys.path.append('/home/dnash/repos/mclimate_tool_cw3e')
from plot_four_panel_fig import output_compare_mclimate_to_reforecast


In [ ]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
import xarray as xr
def load_reforecast(date, varname, F=None):
    path_to_data = '/expanse/nfs/cw3e/cwp140/'
    ## load all F values
    fname_pattern = path_to_data + 'preprocessed/GEFSv12_reforecast/{0}/{1}_{0}_F*.nc'.format(varname, date)
    forecast = xr.open_mfdataset(fname_pattern, engine='netcdf4', concat_dim="step", combine='nested')
    forecast  = forecast.sortby("step") # sort by step (forecast lead)
    tmp = forecast.step[1::2].values
    forecast = forecast.sel(step=tmp) ## select every 6 hours up to 10 days lead time
    step_vals = forecast.step.values / pd.Timedelta(hours=1)
    forecast = forecast.assign_coords({"step": step_vals.astype(int)})
    if F == None:
        forecast = forecast
    else:
        ## load specific F 
        forecast = forecast.sel(step=F)
    print(forecast)
    if varname == 'ivt':
        forecast = forecast.rename({'longitude': 'lon', 'latitude': 'lat', 'time': 'init_date'}) # need to rename this to match GEFSv12 Reforecast mclimate
        # forecast = forecast.drop_vars(["ivtu", "ivtv"])
    elif varname == 'uv1000':
        forecast = forecast.rename({'longitude': 'lon', 'latitude': 'lat'}) # need to rename this to match GEFSv12 Reforecast
        uv = np.sqrt(forecast.u**2 + forecast.v**2)
        if F == None:
            forecast = forecast.assign(uv=(['number', 'step', 'lat','lon'],uv.data))
        else:
            forecast = forecast.assign(uv=(['number', 'lat','lon'],uv.data))
        # forecast = forecast.drop_vars(["u", "v"])
        forecast = forecast.assign_coords(init_date=(pd.to_datetime(date, format='%Y%m%d')))
    else:
        forecast = forecast.assign_coords(init_date=(pd.to_datetime(date, format='%Y%m%d')))
        
    forecast = forecast.mean('number', skipna=True) # ensemble mean
    forecast = forecast.sel(lon=slice(-179.5, -110.), lat=slice(70., 10.))
    
    forecast = forecast.load()

    return forecast

In [ ]:
fdate = '20150813'
varname = 'freezing_level'
F = None
forecast = load_reforecast(fdate, varname, F)
ds = forecast.sel(step=174)
# ds = forecast.sel(step=102, number=[0, 1, 3, 4]) ## ens 2 has no data?
# ds = ds.mean('number') # ensemble mean
ds['freezing_level'] = ds['freezing_level']*3.281 # convert to feet
ds

In [ ]:
ext = [-180., -120., 20, 70]
# Set up projection
# mapcrs = ccrs.Mercator()
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
lats = ds.lat.values
lons = ds.lon.values
dx = np.arange(lons.min().round(),lons.max().round()+20,20)
dy = np.arange(lats.min().round(),lats.max().round()+10,10)

# Create figure
fig = plt.figure(figsize=(5, 5))
fig.dpi = 300
fname = path_to_figs + 'test'
fmt = 'png'

nrows = 1
ncols = 1


# Set up Axes Grid
axes_class = (GeoAxes,dict(projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.2,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.05, cbar_size='3%',label_mode='keep')

kw_clabels = {'fontsize': 7, 'inline': True, 'inline_spacing': 7, 'fmt': '%i',
                  'rightside_up': True, 'use_clabeltext': True}

for k, ax in enumerate(axgr):

    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=True, right_lats=False, bottom_lons=True)
    # ax.set_global()
    # Contour Filled
    lats = ds.lat.values
    lons = ds.lon.values
    clevs = np.arange(0., 60000., 2000.)
    data = ds['freezing_level'].values
    cs = ax.contour(lons, lats, data, transform=datacrs,
                         levels=clevs, colors='k',
                         linewidths=0.75, linestyles='solid')
    plt.clabel(cs, **kw_clabels)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)

# Show
plt.show()

In [ ]:
# impact_dates = ['20051124', '20150818', '20201202', '20231120', '20240923']
impact_dates = ['20240923']
impact_dates = pd.to_datetime(impact_dates)
## create list of init dates we need 
## we will run mclimate based on these dates and lead times
## for each impact date
## for initialization date 1-7 days before impact date
## F000, F024, F072, ...
date_lst = []
F_lst = []
impact_date_lst = []
model_lst = []
for i, date in enumerate(impact_dates):
    ## skip 2 events - 20200227, 20200817
    ## the data from GEFS was too hard to download for these dates
    if (date.strftime("%Y%m%d") == '20200227') | (date.strftime("%Y%m%d") == '20200817'):
        pass
    else:
        for j, init_lead in enumerate(np.arange(1, 8)):
            init_date = date - pd.to_timedelta(init_lead, unit='D')
            F_lst.append(init_lead*24)
            date_lst.append(init_date)
            impact_date_lst.append(date)
    
            if init_date.year < 2020:
                model_name = 'GEFSv12_reforecast'
            else:
                model_name = 'GEFS_archive'
    
            model_lst.append(model_name)

d = {'impact_date': impact_date_lst, 'init_date': date_lst, 'F': F_lst, 'model_name': model_lst}
df = pd.DataFrame(d)
df

In [ ]:
for index, row in df.iloc[3:4].iterrows():
    fdate = row.init_date.strftime('%Y%m%d')
    model = row.model_name
    impact_date = row.impact_date.strftime('%Y%m%d')
    print('Running comparison for ...')
    print('... Impact date: {0}'.format(impact_date))
    print('... Initialization date: {0}'.format(fdate))
    print('... Model name: {0}'.format(model))
    output_compare_mclimate_to_reforecast(fdate, model, impact_date)